# Tutorial on inference with SPEXAI

This tutorial explains how you can use SPEXAI to fit observered spectra from a FITS file.

In [ ]:
import numpy as np
from spexai import Fit, TwoTemp, TempDist

## Fitting an One-Temperature Model

Simulated data or real data can be model with a one temperuture model by  ```Fit```

The paramater that are fitted are the 
- Temperature in KeV  ```temp``` (between 0.2 and 10 KeV)
- Metalicity [solar] ```met```  (larger then 0)
- Turbulent velocity in km/sec ```vel``` (between 0 and 600 km/sec)
- Redshift in log10(z) ```logz``` (between -10 and 1)
- Normalisation ```norm``` (between 1e5 and 1e15)

Additional parameter(s)
- Single element ```X``` abundace ratio with respect to Iron ```ZX```, where ```X``` is tha atom number (larger then 0)

The intial guess with there standard diviation of all the parameters that are fitted can be put into the dictiornary of the ```prior```.

In [ ]:
#intial guess and prior for fit
prior = {
        'temp': {'mu': 5,    'sigma': 2},
        'met' : {'mu': 1,    'sigma': .3},
        'vel' : {'mu': 100,  'sigma': 50},
        'logz': {'mu': -5,   'sigma': 2},
        'norm': {'mu': 1e10, 'sigma': 1e9}
        }

The ensemble samper can be intialized with ```nwalkers``` and ```nsteps``` for the walker to go through, ```prior``` indicates the intial values for the parameters and there Gaussion prior in the likelihood, other parameter that can be initialized are the Luminosity Distance in m and the energy interval of the spectrum in KeV (```e_min``` > 0.1; ```e_max``` < 25).

```fdir_nn``` is the file directory to the trained neural networks and should match the path on your device.

In [ ]:
#initialize the
fit = Fit(50, 200, prior, Luminosity_Distance=1e24, fdir_nn='/home/jip/MasterProject/spexai_code/neuralnetworks/')

### Reading in the FITS files

To be able to fit real data the response of the telescope can be read in by the FITS file, for the Response Matrix File (RMF), effective area response file (ARF).

The data used for fitting can be read in from a FITS file of the observed data ```Fit.load_data```, or can be simulated by the model with ```Fit.sim_data(params)``` here ```params``` is a dictornary with the parameter names and there values.

In addition the response files there is a sparse matrix (```make_sparsex```) used for convulation that implements line broadening to the spectra. The speed an accuracy of the line broadening is strongly dependent on the kernel size of the convulolution ```n``` and the default ```n=300```, increasing ```n``` will make the line-broading more accurete but will also segnifcanly impact the speed.

In [ ]:
'''This step can take a long time to run'''
#reading in the response matrix file
fit.combined_model.load_rm('/home/jip/MasterProject/obs234_HP_obs234_all_det_he.rmf')

In [ ]:
#reading in the effective area
fit.combined_model.load_arf('/home/jip/MasterProject/obs234_HP_obs234_all_det_point_he.arf')

In [ ]:
'''This step can take a long time to run'''
#initializing the sparse matrix for the convolution from line-broadening
fit.combined_model.load_sparsematrix_x(n=300)

In [ ]:
#create your own data
params = {'temp':5,'logz':-2, 'vel':100, 'met':1.2, 'norm':1e10, 'Z8':1.2, 'Z14':0.9}
fit.sim_data(params)

__Add aditional parameters__

Single element abundace that differ with respect to the overall metalicity and Iron can be added in as extra parameter(s).
These are writen in the format ```'ZX'``` with ```X``` the atom number of the elements. ```add_prior``` gives the prior and intial  values of the added paramater(s).

In [ ]:
#names of aditional fitted parameters 
add_prior = {'Z8':{'mu':1, 'sigma':.3},'Z14':{'mu':1, 'sigma':.3}}

### Run Ensemble Sampler Fit

Then after ensemble sampler has been fully intialized the data can be fitted with ```Fit.fit_spectra``` making use of ```emcee``` algorithm.
```Fit.fit_spectra``` will also print the integrated autocorrelation time to give an indication of the burn-in time.

In [ ]:
'''This step can take a long time to run'''
np.seterr(all="ignore")#ignore RuntimeWarning error prints
fit.fit_spectra(add_prior=add_prior)

### Evaluating the fit results

The progress of the walker can be visualized in a timeseries giving the parameter values for each walker at each step in the chain.

In [ ]:
fit.plot_timeseries()

The results of the fit can visualized with a cornerplot. ```Fit.cornerplot``` is also able to save the sampled posterior in an .csv format by giving a directory name ```fitdir```.  In this example you discard the first ```100``` steps in the chain ```discard``` and only reading every 15th step ```thin```. The corner plot can also be oveploted with the true values of the parameters by giving a list ```true_values```, where the values shoud be same order as the prior.

In [ ]:
#true_val = [temp,   met,    vel,    logz    norm,   Z8,    Z14]
true_val =  [5,      1,      100,    -2,     1e10,   1.2,   0.9]

fit.cornerplot(1, thin=1, fitdir=None, true_values=true_val)

The fit results can be directly compared to the data by overplotting the fitted model with parameters drawn from the posterior to the observed data, the number of samples it overplots is given by ```nsample```. Also prints the mean and 1-sigma interval of the fitted parameters.

In [ ]:
'''This step can take a long time to run'''
fit.plot_spectrum(nsample=20)

## Fitting a Two-Temperature Model

Simulated data or real data can be model with a one temperuture model by  ```TwoTemp``` a subclass of Fit

The paramater that are fitted are the 
- First temperature in KeV  ```temp1``` (between 0.2 and 10 KeV)
- Second temperature in KeV  ```temp2``` (between 0.2 and 10 KeV)
- Metalicity [Z_solar] ```met```  (larger then 0)
- Turbulent velocity in km/sec ```vel``` (between 0 and 600 km/sec)
- Redshift in log10(z) ```logz``` (between -10 and 1)
- First normalisation ```norm1``` (between 1e5 and 1e15)
- Second normalisation ```norm2``` (between 1e5 and 1e15)

Additional parameter(s)
- Single element ```X``` abundace ratio with respect to Iron ```ZX```, where ```X``` is tha atom number (larger then 0)

The intial guess with there standard diviation of all the parameters that are fitted can be put into the dictiornary of the ```prior```.

In [ ]:
#intial guess and prior for fit
prior = {
        'temp1': {'mu': 4.5,    'sigma': 2},
        'temp2': {'mu': 6,    'sigma': 2},
        'met' : {'mu': 1,    'sigma': .3},
        'vel' : {'mu': 100,  'sigma': 50},
        'logz': {'mu': -5,   'sigma': 2},
        'norm1': {'mu': 1e10, 'sigma': 1e9},
        'norm2': {'mu': 1.5e10, 'sigma': 1e9}
        }

The ensemble samper can be intialized with ```nwalkers``` and ```nsteps``` for the walker to go through, ```prior``` indicates the intial values for the parameters and there Gaussion prior in the likelihood, other parameter that can be initialized are the Luminosity Distance in m and the energy interval of the spectrum in KeV (```e_min``` > 0.1; ```e_max``` < 25).

```fdir_nn``` is the file directory to the trained neural networks and should match the path on your device.

In [ ]:
#initialize the
fit_2t = TwoTemp(50, 200, prior, Luminosity_Distance=1e24, fdir_nn='/home/jip/MasterProject/spexai_code/neuralnetworks/')

### Reading in the FITS files

To be able to fit real data the response of the telescope can be read in by the FITS file, for the Response Matrix File (RMF), effective area response file (ARF).

The data used for fitting can be read in from a FITS file of the observed data ```TwoTemp.load_data```, or can be simulated by the model with ```TwoTemp.sim_data(params)``` here ```params``` is a dictornary with the parameter names and there values.

In addition the response files there is a sparse matrix (```make_sparsex```) used for convulation that implements line broadening to the spectra. The speed an accuracy of the line broadening is strongly dependent on the kernel size of the convulolution ```n``` and the default ```n=300```, increasing ```n``` will make the line-broading more accurete but will also segnifcanly impact the speed.

In [ ]:
'''This step can take a long time to run'''
#reading in the response matrix file
fit_2t.combined_model.load_rm('/home/jip/MasterProject/obs234_HP_obs234_all_det_he.rmf')

In [ ]:
#reading in the effective area
fit_2t.combined_model.load_arf('/home/jip/MasterProject/obs234_HP_obs234_all_det_point_he.arf')

In [ ]:
'''This step can take a long time to run'''
#initializing the sparse matrix for the convolution from line-broadening
fit_2t.combined_model.load_sparsematrix_x(n=300)

In [ ]:
#create your own data
params = {'temp1':4, 'temp2':5.8, 'logz':-2, 'vel':100, 'met':1.2, 'norm1':8e9, 'norm2':1.2e10}
fit_2t.sim_data(params)

__Add aditional parameters__

Single element abundace that differ with respect to the overall metalicity and Iron can be added in as extra parameter(s).
These are writen in the format ```'ZX'``` with ```X``` the atom number of the elements. ```add_prior``` gives the prior and intial  values of the added paramater(s).

In [ ]:
#names of aditional fitted parameters 
add_prior = {'Z8':{'mu':1, 'sigma':.3},'Z14':{'mu':1, 'sigma':.3}}

### Run Ensemble Sampler Fit

Then after ensemble sampler has been fully intialized the data can be fitted with ```Fit.fit_spectra``` making use of ```emcee``` algorithm.
```TwoTemp.fit_spectra``` will also print the integrated autocorrelation time to give an indication of the burn-in time.

In [ ]:
'''This step can take a long time to run'''
np.seterr(all="ignore")#ignore RuntimeWarning error prints
fit_2t.fit_spectra(add_prior=add_prior)

### Evaluating the fit results

The progress of the walker can be visualized in a timeseries giving the parameter values for each walker at each step in the chain.

In [ ]:
fit.plot_timeseries()

The results of the fit can visualized with a cornerplot. ```TwoTemp.cornerplot``` is also able to save the sampled posterior in an .csv format by giving a directory name ```fitdir```.  In this example you discard the first ```100``` steps in the chain ```discard``` and only reading every 15th step ```thin```. The corner plot can also be oveploted with the true values of the parameters by giving a list ```true_values```, where the values shoud be same order as the prior.

In [ ]:
#true_val = [temp1, temp2,  met,    vel,    logz    norm1,  norm2,  Z8,    Z14]
true_val =  [4,     5.8,    1,      100,    -2,     8e9,    1.2e10, 1.2,   0.9]

fit.cornerplot(1, thin=1, fitdir=None, true_values=true_val)

The fit results can be directly compared to the data by overplotting the fitted model with parameters drawn from the posterior to the observed data, the number of samples it overplots is given by ```nsample```. Also prints the mean and 1-sigma interval of the fitted parameters.

In [ ]:
'''This step can take a long time to run'''
fit.plot_spectrum(nsample=20)

## Fitting a Multi-Temperature Distribution Model

This uses a Subclass of ```Fit```, ```TempDist``` to fit a model with a multi-temperature distribution.

The temperature need to give a function that outputs a temperature grid and temperature distribution described by parameters. The output of the temperature grid should always be inbetween 0.2 and 10 KeV and linearly spaced and the distribution should always be normalized.

In this exapmle we look at a Normal Distribution for our temperature distribution, parametrized by a mean temperature and standard devitation.

The paramater that are fitted
- Mean Temperature in KeV ```mean_temp``` (between 0.2 and 10)
- Standard divation of Temperature in log(KeV) ```log_sd_temp``` (between -5, 0.3)
- Metalicity [Z_solar] ```met```  (larger then 0)
- Turbulent velocity in km/sec ```vel``` (between 0 and 600 km/sec)
- Redshift in log10(z) ```logz``` (between -10 and 1)
- Normalisation ```norm``` (between 1e5 and 1e15)

Additional parameter(s)
- Single element ```X``` abundace ratio with respect to Iron ```ZX```, where ```X``` is tha atom number (larger then 0)

The intial guess with there standard diviation can be put into the dictiornary of the ```prior```.

In [ ]:
def normal_dist(params):
    '''
    Normal Distribution with stdev in logspace
    Parameters
    ----------
    params: dict
        dictionary with the keys of distribution parameters
        in this case 'mean_temp' and 'log_sd_temp'
    '''
    #intialize temperature grid
    low = max(params['mean_temp']-5*10**params['log_sd_temp'], 0.2)
    high = min(params['mean_temp']+5*10**params['log_sd_temp'], 10)
    temp_grid = np.linspace(low, high,  500)
    
    #Gaussion distribution
    mean = params['mean_temp']
    sd = 10**params['log_sd_temp']
    temp_dist = 1/(np.sqrt(2*np.pi)*sd**2) * np.exp(-0.5*((temp_grid-mean)/sd)**2)
    
    #normalize the function 
    temp_dist = temp_dist/np.sum(temp_dist*np.mean(np.diff(temp_grid)))
    return temp_grid, temp_dist

#define allowed interval of parameter of the distribution
interval = {
            'mean_temp': [0.2,10],
            'log_sd_temp'  : [-5, 0.3]
           }

The intial guess with there standard diviation of all the parameters that are fitted can be put into the dictiornary of the ```prior```.

In [ ]:
#intial guess and prior for temp run
prior = {
        'mean_temp': {'mu': 5, 'sigma': 2},
        'log_sd_temp'  : {'mu': -5, 'sigma': 2},
        'met'      : {'mu': 1, 'sigma': .3},
        'vel'      : {'mu': 100, 'sigma': 50},
        'logz'     : {'mu': -5,  'sigma': 2},
        'norm'     : {'mu': 1e10, 'sigma': 1e9}
        }

The ensemble samper can be intialized with ```nwalkers``` and ```nsteps``` for the walker to go through, the prior ```prior```, for the TempDist we need to give the function that returns a normalized temperature distribution with the temperature grid ```normal_dist``` and the bound interval of this ditributions parameter ```interval```, other parameter that can be initialized are the Luminosity Distance in m and the energy interval of the spectrum in KeV (```e_min, e_max```).

In [ ]:
#initialize the
fit_dist = TempDist(50, 200, prior, normal_dist, interval, Luminosity_Distance=9.461e24, e_min=2, e_max=9, fdir_nn='/home/jip/MasterProject/spexai_code/neuralnetworks/')

### Reading in the FITS files

To be able to fit real data the response of the telescope can be read in by the FITS file, for the Response Matrix File (RMF), effective area response file (ARF) and the FITS file of the observed data.

In addition the response files there is a sparse matrix (```make_sparsex```) used for convulation that implements line broadening to the spectra. The speed an accuracy of the line broadening is strongly dependent on the kernel size of the convulolution ```n``` and the default ```n=300```, increasing ```n``` will make the line-broading more accurete but will also segnifcanly impact the speed.

In [ ]:
'''This step can take a long time to run'''
#reading in the response matrix file
fit_dist.combined_model.load_rm('/home/jip/MasterProject/obs234_HP_obs234_all_det_he.rmf')

In [ ]:
#reading in the effective area
fit_dist.combined_model.load_arf('/home/jip/MasterProject/obs234_HP_obs234_all_det_point_he.arf')

In [ ]:
'''This step can take a long time to run'''
#initializing the sparse matrix for the convolution from line-broadening
fit_dist.combined_model.load_sparsematrix_x(n=300)

Now instead of simulating the data we are reading it in from a FITS file

In [ ]:
#reading in the data
fit_dist.load_data('/home/jip/MasterProject/obs234_HP_obs234_all_det_valign_gcor_he.pi')


#### Add aditional parameters

Single element abundace that differ with respect to the overall metalicity and Iron can be added in as extra parameter(s).
These are writen in the format ```'ZX'``` with ```X``` the atom number of the elements. ```add_prior``` gives the prior and intial  values of the added paramater(s).

In [ ]:
#names of aditional fitted parameters 
add_prior = {'Z8':{'mu':1, 'sigma':.3},'Z14':{'mu':1, 'sigma':.3}}

Then after ensemble sampler has been fully intialized the data can be fitted with ```TempDist.fit_spectra``` making use of ```emcee``` algorithm.
```TempDist.fit_spectra``` will aslo print the the integrated autocorrelation time to give an indication of the burn-in time.

In [ ]:
'''This step can take a long time to run'''
np.seterr(all="ignore")#ignore RuntimeWarning error prints
fit_dist.fit_spectra(add_prior=add_prior)

### Evaluating the fit results

The progress of the walker can be visualized in a timeseries giving the parameter values for each walker at each step in the chain.

In [ ]:
fit_dist.plot_timeseries()

The results of the fit can visualized with a cornerplot. ```TempDist.cornerplot``` is also able to save the sampled posterior in an .csv format by giving a directory name ```fitdir```.  In this example you discard the first ```100``` steps in the chain ```discard``` and only reading every 15th step ```thin```. The corner plot can also be oveploted with the true values of the parameters by giving a list ```true_values```, where the values shoud be same order as the prior.

In [ ]:
#true_val = mean_temp,  log_sd_temp,    met,    vel,    logz    norm,   Z8, Z14
true_val = [5,          -2,             1,      100,    -2,     1e10,   1,  1  ]

fit_dist.cornerplot(1, thin=1, fitdir=None, true_values=None)

The fit results can be directly compared to the data by overplotting the fitted model with parameters drawn from the posterior to the observed data, the number of samples it overplots is given by ```nsample```. Also prints the mean and 1-sigma interval of the fitted parameters.

In [ ]:
'''This step can take a long time to run'''
fit_dist.plot_spectrum(nsample=20)